# NN Cylinder

Neural network that is capable of finding the volume of a cylinder given the radius of its base (r) and its height (h).

In [1]:
import math
import random
import pandas as pd
import numpy as np
import shutil

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.INFO)
print(tf.__version__)

1.12.0


Simulate the necessary training dataset.

Assume that the radius and height of the cylinder are both in the range 0.5 to 2.0 and measurements of r, h and V are all rounded off to the nearest 0.1

In [2]:
def rnd_volume_fn(ex=10000):
    volumes = np.empty((ex, 3), float)
    for i in range(ex):
        r = random.uniform(0.5, 2.0)
        h = random.uniform(0.5, 2.0)
        vol = math.pi * h * (r**2)
        volumes[i,0] = round(r, 1)
        volumes[i,1] = round(h, 1)
        volumes[i,2] = round(vol, 1)        
    df = pd.DataFrame(data=volumes, columns=['r','h','V'])
    return df

def split(df):
    msk = np.random.rand(len(df)) < 0.8
    df_train = df[msk]
    df_eval = df[~msk]
    return df_train, df_eval

Train and eval input functions to read from Pandas Dataframe

In [3]:
FEATURES = ['r', 'h']
LABEL = 'V'

nbre_samples = 100000
vol_train, vol_eval = split(rnd_volume_fn(nbre_samples))

def make_train_input_fn(df, num_epochs):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = df[LABEL],
    batch_size = 128,
    num_epochs = num_epochs,
    shuffle = True,
    queue_capacity = 1000
  )


def make_eval_input_fn(df):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = df[LABEL],
    batch_size = 128,
    shuffle = False,
    queue_capacity = 1000
  )

#Feature columns
def make_feature_cols():
  input_columns = [tf.feature_column.numeric_column(k) for k in FEATURES]
  return input_columns

Deep neural network regression

In [7]:
def train_and_evaluate(df_train, df_eval, output_dir):
    estimator = tf.estimator.DNNRegressor(hidden_units = [32, 16, 8, 2],
                                          feature_columns = make_feature_cols(),
                                          model_dir = output_dir)
    
    train_spec=tf.estimator.TrainSpec(input_fn = make_train_input_fn(df_train, num_epochs = 100))
    
    eval_spec=tf.estimator.EvalSpec(input_fn = make_eval_input_fn(df_eval),
                                    steps = None,
                                    start_delay_secs = 1, # start evaluating after N seconds
                                    throttle_secs = 5)  # evaluate every N seconds
                                    
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)


In [8]:
# Run training
OUTDIR = 'vol_trained'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time
tf.summary.FileWriterCache.clear() # ensure filewriter cache is clear for TensorBoard events file

train_and_evaluate(vol_train, vol_eval, OUTDIR)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'vol_trained', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0xb38f5acc0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start tr

INFO:tensorflow:global_step/sec: 274.137
INFO:tensorflow:loss = 17.997765, step = 6901 (0.365 sec)
INFO:tensorflow:global_step/sec: 292.962
INFO:tensorflow:loss = 16.19561, step = 7001 (0.342 sec)
INFO:tensorflow:global_step/sec: 280.955
INFO:tensorflow:loss = 15.356987, step = 7101 (0.356 sec)
INFO:tensorflow:global_step/sec: 319.702
INFO:tensorflow:loss = 19.511227, step = 7201 (0.312 sec)
INFO:tensorflow:global_step/sec: 281.787
INFO:tensorflow:loss = 24.076546, step = 7301 (0.355 sec)
INFO:tensorflow:global_step/sec: 287.201
INFO:tensorflow:loss = 17.456474, step = 7401 (0.348 sec)
INFO:tensorflow:global_step/sec: 285.321
INFO:tensorflow:loss = 17.643082, step = 7501 (0.351 sec)
INFO:tensorflow:global_step/sec: 336.035
INFO:tensorflow:loss = 16.433489, step = 7601 (0.299 sec)
INFO:tensorflow:global_step/sec: 323.272
INFO:tensorflow:loss = 16.809158, step = 7701 (0.308 sec)
INFO:tensorflow:global_step/sec: 285.713
INFO:tensorflow:loss = 18.024696, step = 7801 (0.350 sec)
INFO:tensor

INFO:tensorflow:global_step/sec: 285.655
INFO:tensorflow:loss = 16.850637, step = 15201 (0.350 sec)
INFO:tensorflow:global_step/sec: 285.75
INFO:tensorflow:loss = 14.61267, step = 15301 (0.350 sec)
INFO:tensorflow:global_step/sec: 288.615
INFO:tensorflow:loss = 16.533833, step = 15401 (0.347 sec)
INFO:tensorflow:global_step/sec: 294.12
INFO:tensorflow:loss = 17.847845, step = 15501 (0.340 sec)
INFO:tensorflow:global_step/sec: 293.436
INFO:tensorflow:loss = 14.516688, step = 15601 (0.341 sec)
INFO:tensorflow:global_step/sec: 283.632
INFO:tensorflow:loss = 15.066551, step = 15701 (0.352 sec)
INFO:tensorflow:global_step/sec: 320.574
INFO:tensorflow:loss = 15.233967, step = 15801 (0.312 sec)
INFO:tensorflow:global_step/sec: 289.118
INFO:tensorflow:loss = 20.569525, step = 15901 (0.346 sec)
INFO:tensorflow:global_step/sec: 287.248
INFO:tensorflow:loss = 23.082539, step = 16001 (0.348 sec)
INFO:tensorflow:global_step/sec: 332.999
INFO:tensorflow:loss = 15.26251, step = 16101 (0.300 sec)
INFO

INFO:tensorflow:loss = 17.752401, step = 23401 (0.343 sec)
INFO:tensorflow:global_step/sec: 276.043
INFO:tensorflow:loss = 17.118706, step = 23501 (0.363 sec)
INFO:tensorflow:global_step/sec: 266.596
INFO:tensorflow:loss = 13.830139, step = 23601 (0.375 sec)
INFO:tensorflow:global_step/sec: 297.908
INFO:tensorflow:loss = 16.942022, step = 23701 (0.337 sec)
INFO:tensorflow:global_step/sec: 292.959
INFO:tensorflow:loss = 19.271791, step = 23801 (0.341 sec)
INFO:tensorflow:global_step/sec: 297.914
INFO:tensorflow:loss = 18.340889, step = 23901 (0.336 sec)
INFO:tensorflow:global_step/sec: 276.316
INFO:tensorflow:loss = 13.840907, step = 24001 (0.363 sec)
INFO:tensorflow:global_step/sec: 303.053
INFO:tensorflow:loss = 20.26795, step = 24101 (0.329 sec)
INFO:tensorflow:global_step/sec: 291.009
INFO:tensorflow:loss = 23.655296, step = 24201 (0.343 sec)
INFO:tensorflow:global_step/sec: 284.24
INFO:tensorflow:loss = 16.0113, step = 24301 (0.352 sec)
INFO:tensorflow:global_step/sec: 341.465
INFO

INFO:tensorflow:global_step/sec: 292.263
INFO:tensorflow:loss = 15.728588, step = 31701 (0.341 sec)
INFO:tensorflow:global_step/sec: 347.238
INFO:tensorflow:loss = 15.629339, step = 31801 (0.289 sec)
INFO:tensorflow:global_step/sec: 293.781
INFO:tensorflow:loss = 16.882954, step = 31901 (0.339 sec)
INFO:tensorflow:global_step/sec: 304.678
INFO:tensorflow:loss = 19.054821, step = 32001 (0.328 sec)
INFO:tensorflow:global_step/sec: 296.533
INFO:tensorflow:loss = 20.15164, step = 32101 (0.337 sec)
INFO:tensorflow:global_step/sec: 311.794
INFO:tensorflow:loss = 17.368725, step = 32201 (0.321 sec)
INFO:tensorflow:global_step/sec: 344.945
INFO:tensorflow:loss = 16.991535, step = 32301 (0.291 sec)
INFO:tensorflow:global_step/sec: 315.553
INFO:tensorflow:loss = 17.769537, step = 32401 (0.316 sec)
INFO:tensorflow:global_step/sec: 303.534
INFO:tensorflow:loss = 18.126759, step = 32501 (0.329 sec)
INFO:tensorflow:global_step/sec: 310.735
INFO:tensorflow:loss = 16.94311, step = 32601 (0.322 sec)
IN

INFO:tensorflow:loss = 17.214657, step = 39901 (0.338 sec)
INFO:tensorflow:global_step/sec: 343.186
INFO:tensorflow:loss = 17.765327, step = 40001 (0.291 sec)
INFO:tensorflow:global_step/sec: 312.892
INFO:tensorflow:loss = 14.979846, step = 40101 (0.321 sec)
INFO:tensorflow:global_step/sec: 292.541
INFO:tensorflow:loss = 18.045696, step = 40201 (0.341 sec)
INFO:tensorflow:global_step/sec: 300.426
INFO:tensorflow:loss = 13.082676, step = 40301 (0.333 sec)
INFO:tensorflow:global_step/sec: 322.522
INFO:tensorflow:loss = 19.624508, step = 40401 (0.311 sec)
INFO:tensorflow:global_step/sec: 284.558
INFO:tensorflow:loss = 14.969055, step = 40501 (0.351 sec)
INFO:tensorflow:global_step/sec: 356.355
INFO:tensorflow:loss = 12.734675, step = 40601 (0.282 sec)
INFO:tensorflow:global_step/sec: 298.307
INFO:tensorflow:loss = 16.244022, step = 40701 (0.334 sec)
INFO:tensorflow:global_step/sec: 299.721
INFO:tensorflow:loss = 17.005852, step = 40801 (0.334 sec)
INFO:tensorflow:global_step/sec: 298.525


INFO:tensorflow:global_step/sec: 268.074
INFO:tensorflow:loss = 17.191225, step = 48201 (0.373 sec)
INFO:tensorflow:global_step/sec: 264.966
INFO:tensorflow:loss = 16.476212, step = 48301 (0.378 sec)
INFO:tensorflow:global_step/sec: 295.249
INFO:tensorflow:loss = 15.038738, step = 48401 (0.338 sec)
INFO:tensorflow:global_step/sec: 283.465
INFO:tensorflow:loss = 18.619766, step = 48501 (0.354 sec)
INFO:tensorflow:global_step/sec: 255.991
INFO:tensorflow:loss = 11.518897, step = 48601 (0.389 sec)
INFO:tensorflow:global_step/sec: 292.482
INFO:tensorflow:loss = 15.7738, step = 48701 (0.342 sec)
INFO:tensorflow:global_step/sec: 265.087
INFO:tensorflow:loss = 17.400396, step = 48801 (0.377 sec)
INFO:tensorflow:global_step/sec: 266.51
INFO:tensorflow:loss = 18.130098, step = 48901 (0.375 sec)
INFO:tensorflow:global_step/sec: 258.628
INFO:tensorflow:loss = 13.995415, step = 49001 (0.387 sec)
INFO:tensorflow:global_step/sec: 274.613
INFO:tensorflow:loss = 14.718586, step = 49101 (0.365 sec)
INF

INFO:tensorflow:loss = 17.617252, step = 56401 (0.348 sec)
INFO:tensorflow:global_step/sec: 294.372
INFO:tensorflow:loss = 22.299572, step = 56501 (0.339 sec)
INFO:tensorflow:global_step/sec: 275.389
INFO:tensorflow:loss = 18.46502, step = 56601 (0.363 sec)
INFO:tensorflow:global_step/sec: 310.966
INFO:tensorflow:loss = 15.058186, step = 56701 (0.321 sec)
INFO:tensorflow:global_step/sec: 276.256
INFO:tensorflow:loss = 15.458819, step = 56801 (0.362 sec)
INFO:tensorflow:global_step/sec: 277.738
INFO:tensorflow:loss = 13.5784445, step = 56901 (0.360 sec)
INFO:tensorflow:global_step/sec: 344.042
INFO:tensorflow:loss = 17.307423, step = 57001 (0.291 sec)
INFO:tensorflow:global_step/sec: 315.606
INFO:tensorflow:loss = 15.523596, step = 57101 (0.316 sec)
INFO:tensorflow:global_step/sec: 290.016
INFO:tensorflow:loss = 20.22015, step = 57201 (0.345 sec)
INFO:tensorflow:global_step/sec: 308.403
INFO:tensorflow:loss = 12.840342, step = 57301 (0.324 sec)
INFO:tensorflow:global_step/sec: 274.624
I

In [9]:
estimator

NameError: name 'estimator' is not defined